1. Clone the repo
2. For each file, send queries to convert with both prompts
3. Save the reponses in their own Rust code files
4. Zip those files for download. It'll be easier this way than attempting to
   figure out how to get Colab to do a git commit.

In [20]:
!cd /content
!rm -rf CSCI-5362-Term-Project
!pwd

!git clone --depth=1 https://github.com/sk3p7ic/CSCI-5362-Term-Project.git

/content
Cloning into 'CSCI-5362-Term-Project'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), done.


In [29]:
import openai
import os

openai.api_key = "lalanahnahdoodoo"

# Function to interact with ChatGPT
def convert_to_rust(c_code, model="gpt-4o-mini", safety = True):

    if safety:
        user_prompt_unsafe = ""
    else:
        user_prompt_unsafe = " while maintaining all memory safety errors if possible"

    user_prompt = f"""
    Please convert the following C code to Rust code{user_prompt_unsafe}:

    ```c
    {c_code}
    ```

    Provide only the complete Rust code.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=4000,
        temperature=0.2
    )
    # Extract the code from the response
    rust_code = response.choices[0].message.content.strip()

    # Remove markdown code blocks if present
    if rust_code.startswith("```rust"):
        rust_code = rust_code.split("```rust", 1)[1]
    elif rust_code.startswith("```"):
        rust_code = rust_code.split("```", 1)[1]
    if rust_code.endswith("```"):
        rust_code = rust_code.rsplit("```", 1)[0]

    return rust_code.strip()

def save_safe_rust_code(rust_code, c_file_path):
    output_dir = "Rust Code_Safe"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    c_filename = os.path.basename(c_file_path)
    rust_filename = os.path.splitext(c_filename)[0] + "_safe" + ".rs"
    output_path = os.path.join(output_dir, rust_filename)
    with open(output_path, 'w') as file:
        file.write(rust_code)

def save_unsafe_rust_code(rust_code, c_file_path):
    output_dir = "Rust Code_Unsafe"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    c_filename = os.path.basename(c_file_path)
    rust_filename = os.path.splitext(c_filename)[0] + "_unsafe" ".rs"
    output_path = os.path.join(output_dir, rust_filename)
    with open(output_path, 'w') as file:
        file.write(rust_code)

# Example usage
if __name__ == "__main__":
    # Check if a file path was provided
    # if len(sys.argv) < 2:
    #     print("Usage: python c_to_rust_converter.py <path_to_c_file>")
    #     sys.exit(1)

    # # Get the C file path from command-line arguments
    # c_file_path = sys.argv[1]

    c_file_path = "CSCI-5362-Term-Project/samples/race.c"

    # Use API key from environment variable or use default
    client = openai.OpenAI(api_key=openai.api_key)

    print(f"Reading C file: {c_file_path}")
    with open(c_file_path, 'r') as file:
        c_code = file.read()

    print("Converting C code to Rust using GPT-4o Mini...")
    rust_code = convert_to_rust(c_code, safety = True)
    print('='*80)
    print("Saving Safe Rust code...")
    print('='*80)
    save_safe_rust_code(rust_code, c_file_path)
    print(rust_code)

    rust_code_unsafe = convert_to_rust(c_code, safety = False)
    print('='*80)
    print("Saving Unsafe Rust code...")
    print('='*80)
    save_unsafe_rust_code(rust_code, c_file_path)
    print(rust_code_unsafe)

    print("Conversion complete!")



Reading C file: CSCI-5362-Term-Project/samples/race.c
Converting C code to Rust using GPT-4o Mini...
Saving Safe Rust code...
use std::sync::{Arc, Mutex};
use std::thread;

const NUM_THREADS: usize = 1_000_000;

fn increment_counter(counter: Arc<Mutex<i32>>) {
    let mut num = counter.lock().unwrap();
    *num += 1;
}

fn main() {
    let counter = Arc::new(Mutex::new(0));
    let mut handles = vec![];

    // Create threads
    for _ in 0..NUM_THREADS {
        let counter_clone = Arc::clone(&counter);
        let handle = thread::spawn(move || {
            increment_counter(counter_clone);
        });
        handles.push(handle);
    }

    // Join threads
    for handle in handles {
        handle.join().unwrap();
    }

    // Print final value of counter
    println!("Final counter value: {}", *counter.lock().unwrap());
}
Saving Unsafe Rust code...
use std::thread;

const NUM_THREADS: usize = 1_000_000;

// Shared variable
static mut COUNTER: i32 = 0;

// Thread function
fn inc